In [ ]:
import os
import sys
from decimal import Decimal
import asyncio

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)


from data_sources.clob import CLOBDataSource
from research_notebooks.dneitor.utils import read_yaml_to_dict

clob = CLOBDataSource()
config = read_yaml_to_dict('configs/dneitor-okx_0.4.yml')
FETCH_CANDLES = False
trading_pairs = [config["trading_pair"] for config in config["markets"]]
CONNECTOR_NAME = config["markets"][0]["connector_name"]
DAYS = 7
INTERVALS = ["15m"]

In [ ]:
BATCH_CANDLES_REQUEST = 3
SLEEP_REQUEST = 2.0


if FETCH_CANDLES:
    number_of_calls = (len(trading_pairs) // BATCH_CANDLES_REQUEST) + 1

    all_candles = {}

    for i in range(number_of_calls):
        print(f"Batch {i + 1}/{number_of_calls}")
        start = i * BATCH_CANDLES_REQUEST
        end = (i + 1) * BATCH_CANDLES_REQUEST
        print(f"Start: {start}, End: {end}")
        end = min(end, len(trading_pairs))
        trading_pairs_batch = trading_pairs[start:end]

        tasks = [clob.get_candles_last_days(
        connector_name=CONNECTOR_NAME,
        trading_pair=trading_pair,
        interval=interval,
        days=DAYS,
        ) for trading_pair in trading_pairs_batch for interval in INTERVALS]

        candles = await asyncio.gather(*tasks)
        candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}
        all_candles.update(candles)
        if i != number_of_calls - 1:
            print(f"Sleeping for {SLEEP_REQUEST} seconds")
            await asyncio.sleep(SLEEP_REQUEST)
    clob.dump_candles_cache(os.path.join(root_path, "data"))
else:
    clob.load_candles_cache(os.path.join(root_path, "data"))

In [ ]:
market_configs = config["markets"]
for market_config in market_configs:
    candle = clob.candles_cache[(market_config["connector_name"], market_config["trading_pair"], "15m")]
    fig = candle.fig("candles")
    for price in market_config['prices']:
        fig.add_hline(y=price, line=dict(color='blue', dash='dash'))
    fig.show()

In [ ]:
from services.backend_api_client import BackendAPIClient
from dotenv import load_dotenv
import os

load_dotenv()
servers = os.environ.get("SERVERS").split(",")

In [ ]:
client = BackendAPIClient(servers[0])
client.add_controller_config(config)

In [ ]:
config_binance = read_yaml_to_dict('configs/dneitor-binance_0.4.yml')
client.add_controller_config(config_binance)

In [ ]:
client.deploy_script_with_controllers(
    bot_name="dneitor",
    controller_configs=["dneitor-binance_0.4.yml"],
    image_name="hummingbot/hummingbot:development"
)

In [ ]:
client.get_accounts_state()